In [2]:
import pandas as pd
from wordfreq import top_n_list
import pyphen
import difflib
import pronouncing
import string
from itertools import zip_longest


ModuleNotFoundError: No module named 'wordfreq'

In [ ]:

def load_german_words(n=500_000):
    # Lade die ersten n deutschen Wörter
    words = top_n_list('de', n=n)[:n]
    # Silbentrennung
    dic = pyphen.Pyphen(lang='de')
    syllables = [dic.inserted(w).split('-') for w in words]
    syllable_counts = [len(s) for s in syllables]
    # DataFrame: Wort, Silbenliste, Silbenanzahl
    df = pd.DataFrame({
        'word': words,
        'syllables': syllables,
        'syllable_count': syllable_counts
    })
    return df

# Initiales DataFrame erstellen
df = load_german_words()

def add_column(func):
    """
    Fügt dem globalen DataFrame df eine neue Spalte hinzu.
    func: Funktion, die auf jedes Wort (df['word']) angewendet wird.
    """
    global df
    new_col = df['word'].apply(func)
    df[new_col.name if hasattr(new_col, 'name') and new_col.name else 'new_col'] = new_col
    return df

# Beispiel: Neue Spalte mit Großschreibung
# df = add_column(lambda w: w.upper())
print(df.shape)  # (500000, 4)

(500000, 3)


In [ ]:
path = "words_pandas"

def load_dataframe(path):
    """
    Lädt das DataFrame von path.csv zurück.
    """
    return pd.read_csv(path + '.csv')

df = load_dataframe(path)

In [ ]:
path = "words_pandas"
df.to_csv(path + ".csv", index=False)

In [ ]:
def rhyme_score(syl1: str, syl2: str) -> float:
    """
    Berechnet einen Rhyme-Score (0.0–1.0) zweier Silben,
    basierend auf ihrer Suffix-Ähnlichkeit.
    Je mehr gleiche Endungen, desto höher der Score.
    """
    # Umkehrung, damit Suffixe zu Präfixen werden
    rev1, rev2 = syl1[::-1], syl2[::-1]
    matcher = difflib.SequenceMatcher(None, rev1, rev2)
    return matcher.ratio()

ryhme = rhyme_score("raus","rausch")
print(ryhme)

0.8


In [ ]:
def get_stress_pattern_en(w):
    # z.B. ["1010"] für "analysis"
    patterns = pronouncing.stresses_for_word(w.lower())
    return patterns[0] if patterns else None

print(get_stress_pattern_en("Geister"))  # "1010" → Betonung auf Silbe 1 und 3
print(get_stress_pattern_en("paralysis"))

10
0100


In [ ]:
from phonemizer import phonemize

def get_stress_pattern_de(word):
    # liefert IPA mit primären Stress‐Markern ˈ
    ipa = phonemize(word,
                    language='de',
                    backend='espeak',
                    with_stress=True,
                    strip=True)
    # z.B. "aˈna.lyː.zə"
    return ipa

print(get_stress_pattern_de("Analyse"))  # z.B. aˈna.lyː.zə
print(get_stress_pattern_de("Deine Mama riecht nach Fisch"))

RuntimeError: espeak not installed on your system

In [ ]:
dir(pronouncing)

['__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '__warningregistry__',
 'cmudict',
 'collections',
 'init_cmu',
 'lookup',
 'parse_cmu',
 'phones_for_word',
 'print_function',
 'pronunciations',
 're',
 'resource_stream',
 'rhyme_lookup',
 'rhymes',
 'rhyming_part',
 'search',
 'search_stresses',
 'stresses',
 'stresses_for_word',
 'syllable_count']

In [ ]:
import pyphen

dic = pyphen.Pyphen(lang='de')

def get_syllables(word: str) -> list[str]:
    # z.B. "dauer" → "dau-er"
    return dic.inserted(word).split('-')

print(get_syllables("dauer"))   # ['dau', 'er']


['dau', 'er']


In [ ]:
pip install pyphen

  Using cached pyphen-0.17.2-py3-none-any.whl.metadata (3.2 kB)
Using cached pyphen-0.17.2-py3-none-any.whl (2.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pyphen
import string

# Initialize the Pyphen hyphenator for German
dic = pyphen.Pyphen(lang='de')

def split_to_syls(line: str) -> list[str]:
    """
    Splits a line marked with '*' at the 8th syllable into a fixed window:
      – 7 syllables before the star (padded with " ")
      – the starred syllable
      – 8 syllables after the star (padded with " ")
    Prints any word that Pyphen cannot hyphenate (no '-' inserted).
    """
    # Remove punctuation except '*'
    symbols = string.punctuation.replace('*', '')
    symbols += "„“-–"
    line_clean = line.translate(str.maketrans('', '', symbols))

    # Check for star marker
    if '*' not in line_clean:
        #print("Keine Stern-Markierung gefunden.")
        words = line_clean.strip().lower().split()
        syls = []
        
        for w in words:
            hyph = dic.inserted(w)
            #if '-' not in hyph:
                #print(f"Wort nicht getrennte Silben: {w}")
            syls.extend(hyph.split('-'))
        #print(syls)
        ans = []
        #print(len(syls))
        for i in range(16):
            # print(len(syls) - 15  + i)
            if  -1 < len(syls) - 16  + i:
                ans.append(syls[len(syls) - 16  + i])
            else:
                ans.append(" ")
        return ans

    before, after = line_clean.split('*', 1)
    words_pre  = before.strip().lower().split()
    words_post = after.strip().lower().split()

    # Collect syllables before the star
    syls_pre = []
    for w in words_pre:
        hyph = dic.inserted(w)
        #if '-' not in hyph:
            #print(f"Wort nicht getrennte Silben: {w}")
        syls_pre.extend(hyph.split('-'))

    # Collect syllables after the star
    syls_post = []
    for w in words_post:
        hyph = dic.inserted(w)
        #if '-' not in hyph:
            #print(f"Wort nicht getrennte Silben: {w}")
        syls_post.extend(hyph.split('-'))

    # Starred syllable is the first of post
    star_syl = syls_post[0] if syls_post else ""
    rest_post = syls_post[1:] if len(syls_post) > 1 else []

    # Build pre-window of last 7 syllables, padded left
    pre_window = syls_pre[-7:]
    pre_pad = [" "] * (7 - len(pre_window)) + pre_window

    # Build post-window of first 8 syllables, padded left from end
    post_window = rest_post[:8]
    post_pad = [" "] * (8 - len(post_window)) + post_window

    return pre_pad + [star_syl] + post_pad

# Tests
#print(split_to_syls("Ich habe Hun*ger"))
print(split_to_syls("Ich habe Hun*ger. Ich will essen"))
print(split_to_syls("auf dem der Satz „die sollen sich mal benehmen“ als volksverhetzend zu werten ist."))


[' ', ' ', ' ', 'ich', 'ha', 'be', 'hun', 'ger', ' ', ' ', ' ', ' ', 'ich', 'will', 'es', 'sen']
['sol', 'len', 'sich', 'mal', 'be', 'neh', 'men', 'als', 'volks', 'ver', 'het', 'zend', 'zu', 'wer', 'ten', 'ist']


In [ ]:
def split_to_syls_old(line):
    symbols = string.punctuation
    symbols = symbols.replace("*", "")
    # print(symbols)
    for symbol in symbols:
        line = line.replace(symbol, "")
    # line = line.translate(str.maketrans('', '', symbols))
    print(line.lower())
    words = line.lower().split(" ")
    l_syls = []
    # Optional: schnelleres Lookup durch Index
    syl_dict = df.set_index('word')['syllables'].to_dict()
    for w in words:
        if w in syl_dict:
            l_syls.extend(syl_dict[w])
        # Optional: sonst das Wort selbst anhängen oder überspringen
        # else:
        #     l_syls.append(w)
    return l_syls
print(split_to_syls("Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch."))
print(split_to_syls("Ja ich bin immer obenauf *und ihr holt nur die Drogen raus."))

!"#$%&'()+,-./:;<=>?@[\]^_`{|}~
mir geht nicht die kohle aus *bin dauerhaft im mode rausch
['mir', 'geht', 'nicht', 'die', 'koh', 'le', 'aus', 'dau', 'e', 'r', 'haft', 'im', 'mo', 'de', 'rausch']
!"#$%&'()+,-./:;<=>?@[\]^_`{|}~
ja ich bin immer obenauf *und ihr holt nur die drogen raus
['ja', 'ich', 'bin', 'im', 'mer', 'oben', 'auf', 'ihr', 'holt', 'nur', 'die', 'dro', 'gen', 'raus']


In [ ]:
def lines_rhymescore(l1, l2):
    """
    Berechnet den mittleren Rhyme-Score, indem die Silben
    von hinten nach vorne verglichen werden.
    """
    l1_syls = split_to_syls(l1)
    l2_syls = split_to_syls(l2)
    if len(l2_syls) != 16:
        factor = 0.5
    else:
        factor = 1
    # reverse both lists and zip, fehlende mit "" füllen
    pairs = zip_longest(reversed(l1_syls),
                        reversed(l2_syls),
                        fillvalue="")

    scores = [rhyme_score(s1, s2) for s1, s2 in pairs]
    scores = scores[::-1]
    scores[-3] *= 2
    scores[-2] *= 3
    scores[-1] *= 4
    scores[-8] *= 3
    # print(scores)
    return factor*sum(scores) / len(scores) if scores else 0.0
lines_rhymescore("Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.", "Ja ich bin immer obenauf *und ihr holt nur die Drogen raus.")

0.4918154761904762

In [ ]:
zeilen = [
    "Die Nacht entfaltet sanfte Macht!",
    "Der Mond hat still die Welt entfacht?",
    "Die Sehnsucht lächelnd leise lacht.",
    "Und Ruh' auf müde Seelen sacht,"
]
for i in range(len(zeilen)-1):
    print(lines_rhymescore(zeilen[i],zeilen[i+1]))

0.37569444444444444
0.4262896825396825
0.33154761904761904


In [ ]:
api_key = "sk-proj-1UjM1KbantB6u7DWKKyOT3BlbkFJD88rUr7F3L3WY4urarTc"

In [ ]:
import re
class ResultParser:
    def __init__(self):
        self.expressions = [
            r"<json>([\s\S]*?)<\/json>",
            r"```json\s*([\s\S]*?)\s*```",
            r"(\{[\s\S]*?\})",
            r"(\[[\s\S]*?\])",
        ]

    def parse(self, result, key=None):
        raw = result.get("raw", result)
        last_exception = None

        for expression in self.expressions:
            match = re.search(expression, raw, re.DOTALL | re.IGNORECASE)
            if not match:
                continue
            json_candidate = match.group(1).strip()
            try:
                res = self._eval_candidate(json_candidate)
                if isinstance(res, (dict, list)):
                    return res
            except Exception as exc:
                last_exception = exc

        print("Parsing failed", last_exception)
        return {}

    def _eval_candidate(self, candidate):
        safe = (
            candidate.replace("null", "None")
            .replace("true", "True")
            .replace("false", "False")
        )
        return eval(safe)

In [ ]:
import openai
import json
from typing import List


class RhymeAIManager:
    """Manages an LLM to return twenty lines that each continue a given seed line by rhyming."""

    _SCHEMA_EXAMPLE = {
        "lines": [
            f"Fortsetzung der Zeile {i+1}" for i in range(10)
        ]
    }

    def __init__(self):
        self.api_key = "sk-proj-1UjM1KbantB6u7DWKKyOT3BlbkFJD88rUr7F3L3WY4urarTc"        
        openai.api_key = self.api_key
        self.parser = ResultParser()

    def _chat(self, messages: List[dict]) -> str:
        """Wrapper around OpenAI SDK, returns assistant's text."""
        client_class = getattr(openai, "OpenAI", None)
        try:
            if client_class:
                client = openai.OpenAI(api_key=self.api_key)
                res = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    temperature=0.7
                )
                return res.choices[0].message.content  # type: ignore[attr-defined]
            res = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.7
            )
            return res["choices"][0]["message"]["content"]  # type: ignore[index]
        except Exception as exc:
            print("OpenAI chat error: %s", exc)
            raise

    def get_rhymes(self, seed_line: str, context: str) -> List[str]:
        """
        Return exactly twenty lines.
        Each line must be a direct continuation of `seed_line`, rhyme with it,
        and fit the song context.
        """
        prompt = self._build_prompt(seed_line, context)
        l = len(seed_line)
        filler = """
        Du bist ein kreativer Songwriter. Jede Zeile soll *ausschließlich* die vorgegebene Zeile fortsetzen,
        inhaltlich zum Songkontext passen, sich reimen und exakt 16 Silben haben. Markiere den Anfang der achten Silbe mit einen '*'
        Z.B.: 
        'Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.'
        'Ja ich bin immer obenauf *und ihr holt nur die Drogen raus.'
           """
        messages = [
            {"role": "system", "content": (
                filler
            )},
            {"role": "user", "content": prompt}
        ]

        try:
            raw = self._chat(messages)
            parsed = self.parser.parse({"raw": raw}) or {}
            lines = parsed.get("lines")
            #if isinstance(lines, list) and len(lines) >= 20:
            return lines
            # print("Parser returned invalid lines, falling back to empty list.")
        except Exception:
            print("Failed to get rhymes, returning empty list.")
        return []

    def _build_prompt(self, seed_line: str, context: str) -> str:
        """
        Constructs the user prompt: includes song context and seed line.
        Informs model to output exactly 20 lines as JSON.
        """
        lines = [
            f"Song-Kontext:\n{context.strip()}\n",
            f"Seed-Zeile:\n\"{seed_line.strip()}\"\n",
            """Erstelle genau 10 Zeilen, die jeweils direkt an die Seed-Zeile anschließen und sich reimen. 
            Jede Zeile soll exakt 16 Silben haben. Markiere den Anfang der achten Silbe mit einen '*'.
            Achte darauf, dass sich die achte und die letzen Silben reimen.
        Z.B.: 
        'Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.'
        'Ja ich bin immer obenauf *und ihr holt nur die Drogen raus.'
           """
        ]
        lines.append(
            "Gib die Ausgabe als gültiges JSON im Format:\n" +
            json.dumps(self._SCHEMA_EXAMPLE, ensure_ascii=False, indent=2)
        )
        return "\n".join(lines)

rhymer = RhymeAIManager()

In [ ]:
test_lines = rhymer.get_rhymes("Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.", "Hab Spaß beim Rappen")
print(test_lines)

['Ich bin der Star im Rampenlicht. *Mein Style ist das Highlight.', 'Geld fließt in meine Taschen rein. *Mein Erfolg ist nicht klein.', "Designer-Kleidung, teure Schuh'. *Ich leb' im Luxus, im Nu.", 'Mein Lifestyle ist exquisit. *Von oben bis unten fit.', 'Fahre teure Autos, bin cool. *Mein Leben ist wie im Pool.', "Partys, Champagner und viel Glanz. *Leb' das Leben im Tanz.", 'Im Club, VIP, immer dabei. *Mein Leben ist purer Mai.', "Meine Reime sind immer fresh. *Flow so gut, wie es es wär' frisch.", 'Bühnenlicht strahlt so hell. *Mein Rap klingt wie ne Glockenquelle.', 'Publikum feiert, tanzt und lacht. *Mein Sound hat die Macht.', 'Texte aus tiefster Seele hier. *Meine Worte wie ein Tier.', 'Rappen ist mein Lebenselixier. *Meine Stimme wie ein Meteor.', 'Im Studio, Tag und Nacht. *Mein Sound, der die Welt umfacht.', "Im Takt der Musik, ich schwing'. *Mein Herz für den Rap klingt.", "Mit Leidenschaft im Herz *Rap' ich, bis es schmerzt.", 'Mein Flow ist wie ein wildes Pferd. *Ungezähm

In [ ]:
def sort_rhymes_by_score(base, lines):
    scores = []
    for line in lines:
        # print(line)
        score = lines_rhymescore(base, line)
        scores.append(score)
    n = len(scores)
    for i in range(n):
        # Finde das kleinste Element in lst[i:]
        min_idx = i
        for j in range(i+1, n):
            if scores[j] < scores[min_idx]:
                min_idx = j
        # Tausche lst[i] und lst[min_idx]
        scores[i], scores[min_idx] = scores[min_idx], scores[i]
        lines[i], lines[min_idx] = lines[min_idx], lines[i]
    return lines

In [ ]:
base = "Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch."
sorted_lines = sort_rhymes_by_score(base, test_lines)
for line in sorted_lines:
    print_line(line)

          ein      sam      keit     ein      ta       bu      | ich     |          rap      pe       und      die      welt     ist     | frei    | 
 cham     pa       gner     nicht    nur      ma       lz      | mein    |          le       ben      ist      wie      ein      stru    | del     | 
 im       mer      wei      ter      im       mer      stil    | ich     |                   bleib    stark    wie      ein      ru      | del     | 
          mein     le       ben      ist      ein      lauf    | kei     | ne       zeit     für      ne       pau      se       bru     | der     | 
 kei      ne       pau      se       nur      ge       fühl    | mei     |          ne       wor      te       sind     wie      na      | deln    | 
 ein      zig      ar       tig      mein     pro      fil     | ich     | rap      pe       oh       ne       je       zu       ha      | dern    | 
          im       mer      fresh    im       mer      laut    | mein    |                   style  

In [ ]:
def get_sorted_rhymes(base, context):
    lines = rhymer.get_rhymes(base, context)
    sorted_lines = sort_rhymes_by_score(base, lines)
    return sorted_lines

for line in get_sorted_rhymes("Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.", " ")[::-1]:
    print_line(line)

 lass     uns      flie     gen      hoch     hin      aus     | die     |          nacht    ge       hört     nur      uns      kein    | maus    | 
 he       ben      glä      ser      sto      ßen      an      | das     |          le       ben      ist      ein      gro      ßer     | plan    | 
 mit      mir      ver      lier     den      ver      stand   | das     |          le       ben      ist      ein      bun      ter     | strand  | 
 frei     heit     schmeckt nach     aben     teu      er      | in      | je       dem      mo       ment     da       brennt   das     | feu     | 
 die      ser      stadt    da       zählt    der      stil    | wir     |          le       ben      wild     das      ist      kein    | spiel   | 
 ty       bis      die      lich     ter      blit     zen     | um      | mich     dreht    sich     al       les      mei      ne      | hits    | 
 le       be       je       den      tag      im       glanz   | ver     |          lier     nie    

In [ ]:
def print_line(line):
    highlights = {7, 15}
    for i, syl in enumerate(split_to_syls(line)):
        if i in highlights:
            pre = "|"
            post = "|"
        else:
            pre = ""
            post = ""
        print(pre, f"{syl:8.8}", end=post)
    print(" ")

In [ ]:
test_lines = get_sorted_rhymes("Mir geht nicht die Kohle aus. *Bin dauerhaft im Mode Rausch.", "Hab einfach Spaß beim Rappen")[::-1]

In [ ]:
for line in test_lines:
    print_line(line)

 si       g        ner      von      kopf     bis      fuß     | mei     |                   ne       out      fits     sind     ne      | schau   | 
 show     ti       me       ich      geb      nen      klaps   | mein    |                   er       folg     ist      wie      ein     | fluss   | 
                   hab      die      welt     am       zopf    | kei     |                   ne       gren     zen      ich      geh     | raus    | 
          im       mer      dran     im       mer      top     | mei     |          ne       raps     sind     ein      gu       ter     | schmaus | 
 al       les      echt     nichts   ein      fa       ke      | mei     |                   ne       stim     me       wie      ein     | schrei  | 
 auf      der      büh      ne       dem      par      kett    | mein    |                            flow     fließt   wie      der     | mai     | 
          le       be       schnell  le       be       laut    | mei     |          ne       träu   

In [ ]:
print(test_lines)

['Mode ist mehr als nur Kleidung, es ist Kunst und Leidenschaft.', "Fashionista, Trendsetter, ich leb' den Mode-Traum aus.", "Geld ist keine Frage, wenn's um Fashion geht, bin ich bereit.", 'Schuhe, Shirts und Accessoires, mein Style ist immer chick.', "Mit meinem Style begeistere ich, zieh' alle Blicke auf mich raus.", "Auch wenn's teuer ist, es ist mir wert, das ist meine Welt.", 'In meinem Kleiderschrank nur das Beste, Qualität ist mir wichtig.', 'Einzigartig und individuell, niemand kann mit mir sich messen.', 'Hab einfach Spaß beim Rappen, das ist meine Fashion-Mission.', 'Auf dem Laufsteg meines Lebens, jede Show ein großer Applaus.', 'Designerlabels, Luxus pur, ich investiere in die Zeit.', "Ob Paris, Mailand oder New York, überall bin ich zu Haus'.", 'Ich bin der Star auf jeder Party, strahle heller als der Mond.', 'Immer fresh, immer stylisch, hab den neuesten Trend im Blick.', "Trendbewusst und immer up to date, ich bleib' am Puls der Zeit.", 'Streetwear, High Fashion, ich mi

In [6]:
def get_line(line):
    highlights = {7, 15}
    new_line = ""
    syls = split_to_syls(line)
    #print(syls)
    for i, syl in enumerate(syls):
        if i in highlights:
            pre = "|"
            post = "|"
        else:
            pre = ""
            post = ""
        new_line += pre + f"{syl:8.8}" + post
    return new_line

line = "Sie heißt Lisa, wunderschön"
get_line(line)

'                                                        |        |        sie     heißt   li      sa      wun     der     |schön   |'

In [29]:
with open("die_hoffnung.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()
total = 0
new_lines = []
for line in lines:
    total += len(line)
    if line[0] == "[":
        new_lines.append(line)
        continue
    if line == "" or line == "\n":
        new_lines.append(line)
        continue
    new_line = get_line(line) + "\n"
    # print(new_line)
    new_lines.append(new_line)


with open("song_anaysied.txt", "w", encoding='utf-8') as f:
    intro = [f"Zeichen: {total}\n"]
    f.writelines(intro)
    f.writelines(new_lines)

output_lines = []
for line in lines:
    output_lines.append(line.replace("*", ""))

with open("song_output.txt", "w", encoding='utf-8') as f:
    f.writelines(output_lines)

